In [2]:
class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    END = '\033[0m'

def color_print(text, color):
    print(f"{color}{text}{Color.END}")

In [41]:
# 1.1 Functional Requirements - Load Credit Card Database(SQL)
def display_customer_table(json_file_path):
    # pyspark code to read customer json file
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import initcap, lower,concat,lit,regexp_replace
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("Read_Customer_JSON").getOrCreate()
        # Read the JSON file into a DataFrame
        df = spark.read.json(json_file_path)
        df = df.withColumn("CUST_PHONE", concat(lit("337"), df["CUST_PHONE"]))

        formated_df = df.withColumn("ADDRESS", concat(df["APT_NO"], lit(", "), df["STREET_NAME"])) \
                        .withColumn("CUST_PHONE", regexp_replace(df["CUST_PHONE"].cast("string"), "(\\d{3})(\\d{3})(\\d{4})", "($1)$2-$3")) \
                        .withColumn("FIRST_NAME", initcap("FIRST_NAME")) \
                        .withColumn("MIDDLE_NAME", lower("MIDDLE_NAME")) \
                        .withColumn("LAST_NAME", initcap("LAST_NAME"))

        # Show the DataFrame
        formated_df.select("FIRST_NAME", "MIDDLE_NAME", "LAST_NAME", "SSN", "CREDIT_CARD_NO", "ADDRESS", "CUST_CITY", "CUST_STATE", "CUST_COUNTRY", "CUST_ZIP", "CUST_PHONE", "CUST_EMAIL", "LAST_UPDATED").show(formated_df.count(),truncate=False)
    except Exception as e:
        print(e)
    finally:
        # Stop the SparkSession
        spark.stop()
        

In [5]:
def display_branch_table():
    # pyspark to read branch json file
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import initcap, lower,concat,lit,regexp_replace
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("Read_branch_JSON").getOrCreate()

        # Specify the path to the JSON file
        json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json"

        # Read the JSON file into a DataFrame
        df = spark.read.json(json_file_path)
        formated_df = df.withColumn("BRANCH_PHONE", regexp_replace(df["BRANCH_PHONE"].cast("string"), "(\\d{3})(\\d{3})(\\d{4})", "($1)$2-$3"))
     
    # Show the DataFrame
        formated_df.select("BRANCH_CODE","BRANCH_NAME","BRANCH_STREET","BRANCH_CITY","BRANCH_STATE","BRANCH_ZIP","BRANCH_PHONE","LAST_UPDATED").show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [6]:
def display_credit_table():
    # pyspark to read credit json file
    from pyspark.sql import SparkSession
    
    # Create a SparkSession
    spark = SparkSession.builder.appName("Read_credit_JSON").getOrCreate()

    # Specify the path to the JSON file
    json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json"

    # Read the JSON file into a DataFrame
    df = spark.read.json(json_file_path)

    try: 
    # Show the DataFrame
        df.select("TRANSACTION_ID","DAY","MONTH","YEAR","CREDIT_CARD_NO","CUST_SSN","BRANCH_CODE","TRANSACTION_TYPE","TRANSACTION_VALUE").show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    # Stop the SparkSession
    finally:
        spark.stop()


In [7]:
# 1.2 Data Loading into Database
def create_database_in_mysql():
    import mysql.connector
    import json
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
        )
    
    # Execute a SQL query to create the database
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS `creditcard_capstone`")
        conn.commit()
        print("Database created successfully")
    except Exception as e:
        print(f"Error creating database: {e}")
    finally:
    # Close the Spark session
        cursor.close()
        conn.close()

In [8]:
def create_table_in_mysql():
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import initcap, lower,concat,lit,regexp_replace
    try:
    # Create a Spark session
        spark = SparkSession.builder.appName("CreateTableInMySQL").config("spark.jars", r"E:\soft\Spark\spark-3.5.0-bin-hadoop3\jars\mysql-connector-j-8.3.0.jar").getOrCreate()

        # Define the JDBC URL for the MySQL database
        jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"

        df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer.json")
        df2 = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")
        df3 = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")
        df = df.withColumn("CUST_PHONE", concat(lit("337"), df["CUST_PHONE"]))
        df = df.withColumn("CUST_PHONE", regexp_replace(df["CUST_PHONE"].cast("string"), "(\\d{3})(\\d{3})(\\d{4})", "($1)$2-$3")) \
                .withColumn("FIRST_NAME", initcap("FIRST_NAME")) \
                .withColumn("MIDDLE_NAME", lower("MIDDLE_NAME")) \
                .withColumn("LAST_NAME", initcap("LAST_NAME"))        
                
        df2 = df2.withColumn("BRANCH_PHONE", regexp_replace(df2["BRANCH_PHONE"].cast("string"), "(\\d{3})(\\d{3})(\\d{4})", "($1)$2-$3"))

        # Write the DataFrame to the MySQL table 
        df.write.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "CDW_SAPP_CUSTOMER") \
        .option("user", "root") \
        .option("password", "password") \
        .mode("overwrite") \
        .save()
        
        
        df2.write.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "CDW_SAPP_BRANCH") \
        .option("user", "root") \
        .option("password", "password") \
        .mode("overwrite") \
        .save()
        
        df3.write.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
        .option("user", "root") \
        .option("password", "password") \
        .mode("overwrite") \
        .save()
    except Exception as e:
        print(f"Error creating table: {e}")
        
    finally:
    # Close the Spark session
        spark.stop()

In [9]:
# Use this function to modify the tables and set up relationship between table and primary key
def modify_tables():
    import mysql.connector
    import json
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
            database="creditcard_capstone"
        )
    
        # Execute a SQL query
        cursor = conn.cursor()
        
        cursor.execute("""
                        ALTER TABLE cdw_sapp_customer 
                        MODIFY COLUMN CREDIT_CARD_NO VARCHAR(255),
                        ADD PRIMARY KEY (SSN),
                        ADD INDEX credit_card_index (CREDIT_CARD_NO);
                        """)
        conn.commit()
        
        cursor.execute("""
                       ALTER TABLE cdw_sapp_branch
                       ADD PRIMARY KEY (BRANCH_CODE)
                       """)
        conn.commit()
        
        cursor.execute("""
                       ALTER TABLE cdw_sapp_credit_card
                       MODIFY COLUMN CREDIT_CARD_NO VARCHAR(255),
                       ADD PRIMARY KEY (TRANSACTION_ID),
                       ADD CONSTRAINT `branch_fk` FOREIGN KEY (`BRANCH_CODE`) REFERENCES `CDW_SAPP_BRANCH` (`BRANCH_CODE`),
                       ADD CONSTRAINT `credit_card_fk` FOREIGN KEY (CREDIT_CARD_NO) REFERENCES `CDW_SAPP_CUSTOMER`(CREDIT_CARD_NO)
        """)
        conn.commit()    
        

        
    except Exception as e:
        print(f"Error: {e}")
    finally:
    # Close the session
        cursor.close()
        conn.close()

In [10]:
# 2.1 Transaction Details Module
# Prompt the user for a zip code, provide contextual cues for valid input, and verify it is in the correct format.
# 1 a. Prompt the user for a zip code
def prompt_user_zipcode():
    try:
        while True:
            zipcode = input("Enter your zipcode (Enter 5-digit number): ")
            if len(zipcode) == 5 and zipcode.isdigit():
                return int(zipcode)
            else:
                print("Invalid zipcode. Please enter a 5-digit number.")
                
    except Exception as e:
        print(e)

# b. Prompt the user for a month , year
def prompt_user_month():
    try:
        while True:
            month_input = input("Enter your month (Enter number 1 - 12): ")
            if month_input.isdigit():
                if 1 <= int(month_input) <= 12:
                    return int(month_input)
                else:
                    print("Invalid month. Please enter a number between 1 and 12.")
            else:
                print("Invalid Input. Please enter a number between 1 and 12.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

def prompt_user_year():
    try:
        while True:
            year_input = input("Enter your year (Enter 4-digit number): ")
            if len(year_input) == 4 and year_input.isdigit():
                return int(year_input)
            else:
                print("Invalid year. Please enter a 4-digit number.")
    except Exception as e:
        print(e)


In [11]:
# c. Query the data using Spark SQL base on user input zipcode, month, year
def customer_transaction_details(zipcode, month, year):
    from pyspark.sql import SparkSession
    import json
    # Create a Spark session
    try:
        spark = SparkSession.builder.appName("QueryTransactionData").getOrCreate()

        # Load JSON data into DataFrames
        branch_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")
        transaction_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")

        # Load data from MySQL database using JDBC
        jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"
        properties = {
            "user": "root",
            "password": "password",
            "driver": "com.mysql.cj.jdbc.Driver"
        }

        # Query the data using Spark SQL
        branch_df.createOrReplaceTempView("branch")
        transaction_df.createOrReplaceTempView("transaction")
        
        query = f"""
            SELECT branch.BRANCH_CODE, branch.BRANCH_ZIP, transaction.DAY, transaction.MONTH, transaction.YEAR, branch.BRANCH_NAME, branch.BRANCH_STREET, branch.BRANCH_CITY, branch.BRANCH_STATE, branch.BRANCH_PHONE, transaction.CREDIT_CARD_NO, transaction.CUST_SSN, transaction.TRANSACTION_ID, transaction.TRANSACTION_TYPE, transaction.TRANSACTION_VALUE, branch.LAST_UPDATED
            FROM branch 
            JOIN transaction 
            Using (BRANCH_CODE) 
            WHERE branch.BRANCH_ZIP = {zipcode} 
            AND transaction.MONTH = {month} 
            AND transaction.YEAR = {year}
        """
        result_df = spark.sql(query)

        # Show the result DataFrame
        result_df.show()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:  
        # Stop the Spark session
        spark.stop()

In [12]:
# d. Sort the transaction by day
def sort_transaction_by_day():
    # pyspark to read credit json file
    from pyspark.sql import SparkSession
    # Create a SparkSession
    spark = SparkSession.builder.appName("Read_credit_JSON").getOrCreate()

    # Specify the path to the JSON file
    json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json"

    # Read the JSON file into a DataFrame
    df = spark.read.json(json_file_path)

    try: 
    # Show the DataFrame
        df.select("TRANSACTION_ID","DAY","MONTH","YEAR","CREDIT_CARD_NO","CUST_SSN","BRANCH_CODE","TRANSACTION_TYPE","TRANSACTION_VALUE").orderBy("DAY", ascending=False).show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [13]:
# 2 Use to display number and value of transaction type
def transaction_number_value(type):
    from pyspark.sql import SparkSession
    # Create a SparkSession
    spark = SparkSession.builder.appName("trasaction_JSON").getOrCreate()
    # Specify the path to the JSON file
    json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json"
    # Read the JSON file into a DataFrame
    df = spark.read.json(json_file_path)

    try: 
    # Show the DataFrame
        df.createOrReplaceTempView("transaction")
        spark.sql(f"""SELECT TRANSACTION_TYPE, count(TRANSACTION_TYPE) as Number, round(Sum(TRANSACTION_VALUE),2) as Total 
                    FROM transaction 
                    Group By TRANSACTION_TYPE 
                    HAVING TRANSACTION_TYPE = '{type}'""").show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()


In [14]:
def prompt_user_type():
    try:
        while True:
            type = input("Enter Transaction Type (Healthcare, Automotive, etc.): ")
            if type.isalpha():
                return type
            else:
                print("Invalid input. Please enter string only.")
                
    except Exception as e:
        print(e)

In [15]:
# 3. Use to display number and total value of transaction branch by given state
def transaction_branch_number_value_state(state):
    from pyspark.sql import SparkSession
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("trasaction_JSON").getOrCreate()
        # Read the JSON file into a DataFrame
        df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")
        branch_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")

        
        # Show the DataFrame
        df.createOrReplaceTempView("transaction")
        branch_df.createOrReplaceTempView("branch")
        
        query = f"""
            SELECT branch.BRANCH_STATE, count(TRANSACTION_TYPE) as Number, round(Sum(TRANSACTION_VALUE),2) as Total_value_transactions
            FROM branch 
            JOIN transaction 
            Using (BRANCH_CODE)
            WHERE branch.BRANCH_STATE = '{state}'
            Group By branch.BRANCH_STATE
        """
        
        result_df = spark.sql(query)
        # Show the result DataFrame
        result_df.show()
        
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [16]:
def prompt_user_state():
    try:
        while True:
            state = input("Enter State (LA, CA, etc.): ")
            if state.isalpha():
                if len(state) == 2:
                    return state.upper()
                else:
                    print("Invalid input. Please enter 2-letters for the state only.")
            else:
                print("Invalid input. Please enter string only.")
                
    except Exception as e:
        print(e)

In [17]:
# 2.2 Customer Details Module
# 1 Check existing account details of customer
def check_cust_detail(SSN):
    from pyspark.sql import SparkSession
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("check customer detail").getOrCreate()
        # Read the JSON file into a DataFrame
        credit_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")
        customer_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer.json")
        branch_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")

        
        # Show the DataFrame
        credit_df.createOrReplaceTempView("credit")
        customer_df.createOrReplaceTempView("customer")
        branch_df.createOrReplaceTempView("branch")
        
        query = f"""
            SELECT customer.SSN, 
            customer.FIRST_NAME, 
            customer.MIDDLE_NAME, 
            customer.LAST_NAME, 
            customer.CREDIT_CARD_NO, 
            customer.APT_NO, 
            customer.STREET_NAME, 
            customer.CUST_CITY, 
            customer.CUST_STATE, 
            customer.CUST_ZIP, 
            customer.CUST_COUNTRY, 
            customer.CUST_EMAIL, 
            customer.CUST_PHONE,
            branch.BRANCH_NAME,
            credit.DAY,
            credit.MONTH,
            credit.YEAR,
            credit.TRANSACTION_TYPE,
            credit.TRANSACTION_VALUE
            FROM customer
            JOIN credit
            Using (CREDIT_CARD_NO)
            JOIN branch
            Using (BRANCH_CODE)
            WHERE RIGHT(customer.SSN, 4) = '{SSN}'
            ORDER BY credit.YEAR, credit.MONTH, credit.DAY
            """
            
        result_df = spark.sql(query)
        # Show the result DataFrame
        result_df.show()
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [5]:
def prompt_user_SSN():
    try:
        while True:
            SSN = input("Enter Customer last 4 SSN: ")
            if SSN.isnumeric():
                if len(SSN) == 4:
                    return SSN
                else:
                    print("Invalid input. Please enter 4-digit number only.")
            else:
                print("Invalid input. Please enter number only.")
                
    except Exception as e:
        print(e)

In [4]:
def check_cust_SSN(SSN):
    import json
    try:
        with open(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer.json") as f:
            for line in f:
                customer_data = json.loads(line)
                if str(customer_data["SSN"])[-4:] == str(SSN):
                    return True
            return False
    except Exception as e:
        print(e)
    finally:
        f.close()

In [44]:
def write_cust_copy():
    import mysql.connector
    import json
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType
    spark = SparkSession.builder.appName("Write_JSON_File").getOrCreate()
    schema = StructType([
    StructField("FIRST_NAME", StringType(), True),
    StructField("MIDDLE_NAME", StringType(), True),
    StructField("LAST_NAME", StringType(), True),
    StructField("SSN", IntegerType(), True),
    StructField("CREDIT_CARD_NO", StringType(), True),
    StructField("APT_NO", StringType(), True),
    StructField("STREET_NAME", StringType(), True),
    StructField("CUST_CITY", StringType(), True),
    StructField("CUST_STATE", StringType(), True),
    StructField("CUST_COUNTRY", StringType(), True),
    StructField("CUST_ZIP", StringType(), True),
    StructField("CUST_PHONE", StringType(), True),
    StructField("CUST_EMAIL", StringType(), True),
    StructField("LAST_UPDATED", StringType(), True),
    ])
    try:     
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
            database="creditcard_capstone"
        )
          
        query = f"""
                SELECT 
                FIRST_NAME,
                MIDDLE_NAME, 
                LAST_NAME, 
                SSN, 
                CREDIT_CARD_NO, 
                APT_NO,
                STREET_NAME, 
                CUST_CITY, 
                CUST_STATE, 
                CUST_COUNTRY, 
                CUST_ZIP, 
                CUST_PHONE, 
                CUST_EMAIL, 
                LAST_UPDATED 
                FROM cdw_sapp_customer"""
                
        cursor = conn.cursor()        
        cursor.execute(query)
        result = cursor.fetchall()
        
        data = []
        for row in result:
            row_dict = [row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13]]
            data.append(row_dict)
        df = spark.createDataFrame(data, schema=schema)
        # Write the DataFrame to a JSON file
        df.write.mode("overwrite").json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer_copy.json")
        
    except Exception as e:
        print(e)
    finally:
        spark.stop()

In [43]:
# 2) Modify existing account details of a customer
def modify_cust_detail(column, value, SSN):
    import mysql.connector
    import json
    from pyspark.sql import SparkSession
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType
    spark = SparkSession.builder.appName("Write_JSON_File").getOrCreate()
    schema = StructType([
    StructField("FIRST_NAME", StringType(), True),
    StructField("MIDDLE_NAME", StringType(), True),
    StructField("LAST_NAME", StringType(), True),
    StructField("SSN", IntegerType(), True),
    StructField("CREDIT_CARD_NO", StringType(), True),
    StructField("APT_NO", StringType(), True),
    StructField("STREET_NAME", StringType(), True),
    StructField("CUST_CITY", StringType(), True),
    StructField("CUST_STATE", StringType(), True),
    StructField("CUST_COUNTRY", StringType(), True),
    StructField("CUST_ZIP", StringType(), True),
    StructField("CUST_PHONE", StringType(), True),
    StructField("CUST_EMAIL", StringType(), True),
    StructField("LAST_UPDATED", StringType(), True),
    ])
    
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
            database="creditcard_capstone"
        )
    
        # Execute a SQL query
        cursor = conn.cursor()
        
        if check_cust_SSN(SSN):
            cursor.execute(f"""
                            UPDATE cdw_sapp_customer
                            SET {column} = '{value}'
                            WHERE RIGHT(SSN, 4) = '{SSN}';
                        """)
            conn.commit()
        else:
            print("Invalid Customer SSN.")
        
        query = f"""
                SELECT 
                FIRST_NAME,
                MIDDLE_NAME, 
                LAST_NAME, 
                SSN, 
                CREDIT_CARD_NO, 
                APT_NO,
                STREET_NAME, 
                CUST_CITY, 
                CUST_STATE, 
                CUST_COUNTRY, 
                CUST_ZIP, 
                CUST_PHONE, 
                CUST_EMAIL, 
                LAST_UPDATED 
                FROM cdw_sapp_customer"""
                
        cursor.execute(query)
        result = cursor.fetchall()
        
        data = []
        for row in result:
            row_dict = [row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13]]
            data.append(row_dict)
        df = spark.createDataFrame(data, schema=schema)
        # Write the DataFrame to a JSON file
        df.write.mode("overwrite").json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer_copy.json")
        
    except Exception as e:
        print(e)
    finally:
        conn.close()
        spark.stop()

In [21]:
def prompt_user_column():
    try:
        col = ["FIRST_NAME", "MIDDLE_NAME", "LAST_NAME", "CREDIT_CARD_NO", "CUST_CITY", "CUST_STATE", "CUST_COUNTRY", "CUST_ZIP", "CUST_PHONE", "CUST_EMAIL", "APT NO", "STREET_NAME", "SSN"]
        color_print("Columns list: ", Color.GREEN)
        print(col)
        while True:
            column = input("Enter column name to update: ")
            if column in col:
                return column
            else:
                print("Invalid input. Please enter a valid column name.")
    except Exception as e:
        print(e)
    
def prompt_user_value():
    try:
        while True:
            value = input("Enter new value: ")
            if value.isnumeric():
                return int(value)
            else:
                return value
    except Exception as e:
        print(e)

In [47]:
# 3. Generate monthly bill for a credit card number for a given month and year

def check_credit_card(credit_card_no):
    import json
    try:
        with open(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json") as f:
            for line in f:
                credit_data = json.loads(line)
                if str(credit_data["CREDIT_CARD_NO"]) == str(credit_card_no):
                    return True
            return False
    except Exception as e:
        print(e)
    finally:
        f.close()

In [23]:
def generate_monthly_bill(credit_card_no, month, year):
    import json
    from pyspark.sql import SparkSession
    
    spark = SparkSession.builder.appName("trasaction_JSON").getOrCreate()
    try:
        if check_credit_card(credit_card_no):
            bill_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")
            bill_df.createOrReplaceTempView("bill")
            
            query = f"""
                SELECT  CREDIT_CARD_NO, 
                        MONTH,
                        YEAR,
                        TRANSACTION_TYPE, 
                        TRANSACTION_VALUE
                        FROM bill
                        WHERE CREDIT_CARD_NO = {credit_card_no} 
                        AND MONTH = {month} 
                        AND YEAR = {year}              
                        ORDER BY YEAR, MONTH, TRANSACTION_TYPE
                """
            bill_df = spark.sql(query)
            bill_df.show()
            
            total = f"""SELECT  round(sum(TRANSACTION_VALUE),2) as TOTAL_TRANSACTION 
                                FROM bill 
                                WHERE CREDIT_CARD_NO = {credit_card_no} 
                                AND MONTH = {month} 
                                AND YEAR = {year}
                                
                                """
            bill_df = spark.sql(total)
            bill_df.show()
    except Exception as e:
        print(e)
    finally:
        spark.stop()

In [24]:
def prompt_user_credit_card():
    try:
        while True:
            credit_card_no = input("Enter your credit card number (16 digits): ")
            if len(credit_card_no) == 16 and credit_card_no.isdigit():
                return str(credit_card_no)
            else:
                print("Invalid credit card number. Please enter a 16-digit number.")
                
    except Exception as e:
        print(e)

In [45]:
def display_transactions_between_dates(credit_card_no, start_day, start_month, start_year, end_day, end_month, end_year):
    import json
    from pyspark.sql import SparkSession
    
    spark = SparkSession.builder.appName("transactions_JSON").getOrCreate()
    
    try:
        if check_credit_card(credit_card_no):
            transaction_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")
            transaction_df.createOrReplaceTempView("transactions")
            
            query = f"""
                SELECT  CREDIT_CARD_NO, 
                        DAY,
                        MONTH,
                        YEAR,
                        TRANSACTION_TYPE, 
                        TRANSACTION_VALUE
                FROM transactions
                WHERE CREDIT_CARD_NO = '{credit_card_no}' 
                AND DAY >= {start_day}
                AND DAY <= {end_day}
                AND MONTH >= {start_month}
                AND MONTH <= {end_month}
                AND YEAR >= {start_year}
                AND YEAR <= {end_year}
                ORDER BY YEAR DESC, MONTH DESC, DAY DESC
            """
            result_df = spark.sql(query)
            result_df.show(result_df.count(), False)
            
    except Exception as e:
        print(e)
    finally:
        spark.stop()

In [26]:
def prompt_user_day():
    try:
        while True:
            day_input = input("Enter your day (1-31): ")
            if len(day_input) == 1 or len(day_input) == 2 and day_input.isdigit():
                if int(day_input) >= 1 and int(day_input) <= 31:
                    return int(day_input)
                else:
                    print("Invalid day. Please enter a number between 1 and 31.")
            else:
                print("Invalid day. Please enter a number between 1 and 31.")
    except Exception as e:
        print(e)

In [27]:
def display_menu():
    color_print("=====Welcome to the CDW SAPP Credit Card Transaction Management System!=====", Color.BLUE)
    color_print("What do you want to do? Please choose an option: ", Color.GREEN)
    color_print("1. Display Customer Information.", Color.RED)
    color_print("2. Display Branch Information.", Color.RED)
    color_print("3. Display Credit Card and Transaction Information.", Color.RED)
    color_print("4. Retrieve customer's transaction in specified zip code for a given month and year.", Color.RED)
    color_print("5. Sort the transactions by day in descending order.", Color.RED)
    color_print("6. Display number and total values of transactions for a given type.", Color.RED)
    color_print("7. Display the total number and total values of transactions for branches in a given state.", Color.RED)
    color_print("8. Check existing account details of a customer.", Color.RED)
    color_print("9. Modify existing account details of a customer.", Color.RED)
    color_print("10. Generate monthly bill.", Color.RED)
    color_print("11. Display transactions between dates.", Color.RED)
    color_print("12. Exit.", Color.YELLOW)
    color_print("==========================================================================", Color.BLUE)

In [28]:
def prompt_user_choice():
    while True:
        try:
            choice = int(input("Enter your choice: "))
            if 1 <= choice <= 12:
                return choice
            else:
                print("Invalid choice. Please enter a number between 1 and 10.")
        except ValueError:
            print("Invalid input. Please enter a number.")

In [32]:
def execute_option(option):
    import sys
    if option == 1:
        color_print("1. Display Customer Information.", Color.RED)
        display_customer_table()
    
    elif option == 2:
        color_print("2. Display Branch Information.", Color.RED)
        display_branch_table()
        
    elif option == 3:
        color_print("3. Display Credit Card and Transaction Information.", Color.RED)
        display_credit_table()
        
    elif option == 4:
        color_print("4. Retrieve customer's transaction in specified zip code for a given month and year", Color.RED)
        customer_transaction_details(prompt_user_zipcode(), prompt_user_month(), prompt_user_year())
        
    elif option == 5:
        color_print("5. Sort the transactions by day in descending order.", Color.RED)
        sort_transaction_by_day()
        
    elif option == 6:
        color_print("6. Display number and total values of transactions for a given type.", Color.RED)
        transaction_number_value(prompt_user_type())
        
    elif option == 7:
        color_print("7. Display the total number and total values of transactions for branches in a given state.", Color.RED)
        transaction_branch_number_value_state(prompt_user_state())
        
    elif option == 8:
        color_print("8. Check existing account details of a customer.", Color.RED)
        check_cust_detail(prompt_user_SSN())
        
    elif option == 9:
        color_print("9. Modify existing account details of a customer.", Color.RED)
        modify_cust_detail(prompt_user_column(), prompt_user_value(), prompt_user_SSN())
        
    elif option == 10:
        color_print("Generate monthly bill.", Color.RED)
        generate_monthly_bill(prompt_user_credit_card(), prompt_user_month(), prompt_user_year())
        
    elif option == 11:
        color_print("Display transactions between dates.", Color.RED)
        color_print("Please enter the start date: ", Color.GREEN)
        start_day = prompt_user_day()
        color_print("Please enter the start month: ", Color.GREEN)
        start_month = prompt_user_month()
        color_print("Please enter the start year: ", Color.GREEN)
        start_year = prompt_user_year()
        color_print("Please enter the end day: ", Color.GREEN)
        end_day = prompt_user_day()
        color_print("Please enter the end month: ", Color.GREEN)
        end_month = prompt_user_month()
        while True:
            color_print("Please enter the end year: ", Color.GREEN)
            end_year = prompt_user_year()
            if end_year >= start_year:
                break
            else:
                print("Invalid year. Please enter a year greater than or equal to the start year.")
        
    elif option == 12:
        color_print("12. Exit.", Color.YELLOW)
        color_print("Exiting program", Color.YELLOW)
        
        
    else:
        color_print("Invalid option. Please enter a number between 1 and 12.", Color.RED)

In [11]:
def display_cust_Info(SSN):
    from pyspark.sql import SparkSession
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("check customer detail").getOrCreate()
        # Read the JSON file into a DataFrame
        customer_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer.json")       
        # Show the DataFrame
        customer_df.createOrReplaceTempView("customer")
        query = f"""
            SELECT SSN, 
            FIRST_NAME, 
            MIDDLE_NAME, 
            LAST_NAME, 
            CREDIT_CARD_NO, 
            APT_NO, 
            STREET_NAME, 
            CUST_CITY, 
            CUST_STATE, 
            CUST_ZIP, 
            CUST_COUNTRY, 
            CUST_EMAIL, 
            CUST_PHONE
            FROM customer
            WHERE RIGHT(SSN, 4) = {SSN}
            """
        result_df = spark.sql(query)
        # Show the result DataFrame
        result_df.show()
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [33]:
display_menu()

while True:
    option = prompt_user_choice()
    execute_option(option)
    if option == 12:
        break

=====Welcome to the CDW SAPP Credit Card Transaction Management System!=====
What do you want to do? Please choose an option: 
1. Display Customer Information.
2. Display Branch Information.
3. Display Credit Card and Transaction Information.
4. Retrieve customer's transaction in specified zip code for a given month and year.
5. Sort the transactions by day in descending order.
6. Display number and total values of transactions for a given type.
7. Display the total number and total values of transactions for branches in a given state.
8. Check existing account details of a customer.
9. Modify existing account details of a customer.
10. Generate monthly bill.
11. Display transactions between dates.
12. Exit.
12. Exit.
Exiting program
